In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("walmart").getOrCreate()

In [3]:
spark

In [4]:
#loading data 
df = spark.read.csv('/FileStore/tables/walmart/walmart_stock.csv',inferSchema=True,header=True)

In [5]:
df.columns

In [6]:
df.printSchema()

In [7]:
df.head(5)

In [8]:
df.describe().show()

In [9]:
df.describe().printSchema()


In [10]:
from pyspark.sql.functions import format_number
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()


In [11]:
df2 = df.withColumn("HV Ratio",df["High"]/df["Volume"])#.show()
# df2.show()
df2.select('HV Ratio').show()
df2.printSchema()

In [12]:
df.orderBy(df["High"].desc()).head(1)[0][0]

In [13]:
# Also could have gotten this from describe()
from pyspark.sql.functions import mean
df.select(mean("Close")).show()

In [14]:
from pyspark.sql.functions import max,min
df.select(max("Volume"),min("Volume")).show()


In [15]:
#df.filter("Close < 60").count()
df.filter(df['Close'] < 60).count()


In [16]:
#What percentage of the time was the High greater than 80 dollars ?
#In other words, (Number of Days High>80)/(Total Days in the dataset)
(df.filter(df["High"]>80).count()*1.0/df.count())*100


In [17]:
#What is the Pearson correlation between High and Volume?
from pyspark.sql.functions import corr
df.select(corr("High","Volume")).show()


In [18]:
#What is the max High per year?
from pyspark.sql.functions import year
yeardf = df.withColumn("Year",year(df["Date"]))

In [19]:
max_df = yeardf.groupBy('Year').max()


In [20]:
max_df.select('Year','max(High)').show()

In [21]:
#What is the average Close for each Calendar Month?
#In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc...result will have a value for each of these months
from pyspark.sql.functions import month
monthdf = df.withColumn("Month",month("Date"))
monthdf.show()

In [22]:
monthavgs = monthdf.select("Month","Close").groupBy("Month").mean()
monthavgs.show()

In [23]:
monthavgs.select("Month","avg(Close)").orderBy('Month').show()
